# Logistic Regression Customer Churn Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [1]:
import findspark
findspark.init("/home/ratzz/spark-2.4.4-bin-hadoop2.7")
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('lr').getOrCreate()

In [3]:
df = spark.read.csv("customer_churn.csv",inferSchema=True,header = True)

In [12]:
len(df.toPandas()["Company"]) 

900

In [13]:
df.toPandas()["Company"].value_counts() #out of 900 rows 4 rows has same company ,makes company pretty useless feature 

Anderson Group                  4
Wilson PLC                      3
Williams PLC                    3
Jones LLC                       2
Smith and Sons                  2
                               ..
Walker-Baker                    1
Arellano, Smith and Lang        1
Jordan, Guerrero and Hoffman    1
Schaefer-Berry                  1
Stein PLC                       1
Name: Company, Length: 873, dtype: int64

In [14]:
df.toPandas()

,Names,Age,Total_Purchase,Account_Manager,Years,Num_Sites,Onboard_date,Location,Company,Churn
0,Cameron Williams,42.0,11066.80,0,7.22,8.0,2013-08-30 07:00:40,"10265 Elizabeth Mission Barkerburgh, AK 89518",Harvey LLC,1
1,Kevin Mueller,41.0,11916.22,0,6.50,11.0,2013-08-13 00:38:46,"6157 Frank Gardens Suite 019 Carloshaven, RI 1...",Wilson PLC,1
2,Eric Lozano,38.0,12884.75,0,6.67,12.0,2016-06-29 06:20:07,"1331 Keith Court Alyssahaven, DE 90114","Miller, Johnson and Wallace",1
3,Phillip White,42.0,8010.76,0,6.71,10.0,2014-04-22 12:43:12,"13120 Daniel Mount Angelabury, WY 30645-4695",Smith Inc,1
4,Cynthia Norton,37.0,9191.58,0,5.56,9.0,2016-01-19 15:31:15,"765 Tricia Row Karenshire, MH 71730",Love-Jones,1
...,...,...,...,...,...,...,...,...,...,...
895,Paul Miller,42.0,12800.82,1,3.62,8.0,2007-12-01 13:29:34,"9316 Julian Fort Suite 328 North Leslie, ME 43961",Evans-Lucero,0
896,Natalie Hodges,52.0,9893.92,0,6.91,7.0,2008-12-28 15:23:58,"8419 William Square Apt. 695 Martinville, RI 3...",Perry and Sons,0
897,Ana Smith,45.0,12056.18,0,5.46,4.0,2014-06-20 05:10:09,Unit 8633 Box 8738 DPO AA 14126-5026,Schneider-Smith,0
898,Justin Leonard,51.0,6517.93,1,5.47,10.0,2012-05-30 00:15:43,"49800 Torres Ways Suite 886 West Bradleybury, ...",Robles-Abbott,0


In [15]:
#we wont use location and onboard rn and will see later by doing some feature engineering on both

In [16]:
df.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [19]:
data = df.select(['Age',
 'Total_Purchase',
 'Years',
 'Num_Sites','Churn'])

In [20]:
data.toPandas().head()

,Age,Total_Purchase,Years,Num_Sites,Churn
0,42.0,11066.80,7.22,8.0,1
1,41.0,11916.22,6.50,11.0,1
2,38.0,12884.75,6.67,12.0,1
3,42.0,8010.76,6.71,10.0,1
4,37.0,9191.58,5.56,9.0,1


In [25]:
data.printSchema()

root
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Churn: integer (nullable = true)



In [23]:
data.columns

['Age', 'Total_Purchase', 'Years', 'Num_Sites', 'Churn']

In [29]:
from pyspark.ml.feature import StandardScaler,VectorAssembler

In [28]:
# algo_ss = StandardScaler(inputCol=['Age', 'Total_Purchase', 'Years', 'Num_Sites'],outputCol=['Agestd', 'Total_Purchasestd', 'Yearsstd', 'Num_Sitesstd'])

In [32]:
assembler = VectorAssembler(inputCols=['Age', 'Total_Purchase', 'Years', 'Num_Sites']
                           ,outputCol="features")

In [34]:
output = assembler.transform(data)

In [37]:
output = output.select('features','Churn')

In [38]:
output.columns

['features', 'Churn']

In [39]:
ss = StandardScaler(inputCol='features',outputCol="scaled_features")

In [40]:
scaled = ss.fit(output).transform(output)

In [42]:
final_data = scaled.select("scaled_features","Churn").withColumnRenamed("scaled_features",'features')

In [43]:
final_data.show()

+--------------------+-----+
|            features|Churn|
+--------------------+-----+
|[6.85427758232319...|    1|
|[6.69108049702978...|    1|
|[6.20148924114955...|    1|
|[6.85427758232319...|    1|
|[6.03829215585614...|    1|
|[7.83346009408364...|    1|
|[7.18067175291000...|    1|
|[5.22230672938909...|    1|
|[7.01747466761659...|    1|
|[6.52788341173637...|    1|
|[4.89591255880227...|    1|
|[7.34386883820341...|    1|
|[7.34386883820341...|    1|
|[6.52788341173637...|    1|
|[6.69108049702978...|    1|
|[6.20148924114955...|    1|
|[7.34386883820341...|    1|
|[7.01747466761659...|    1|
|[8.64944552055069...|    1|
|[7.50706592349682...|    1|
+--------------------+-----+
only showing top 20 rows



In [44]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [45]:
from pyspark.ml.classification import LogisticRegression

In [46]:
algo_log = LogisticRegression(labelCol="Churn")

In [47]:
log_model = algo_log.fit(train_data)

In [48]:
results_log = log_model.evaluate(test_data)

In [53]:
results_log.roc.show()

+--------------------+-------------------+
|                 FPR|                TPR|
+--------------------+-------------------+
|                 0.0|                0.0|
|                 0.0| 0.0425531914893617|
|                 0.0| 0.0851063829787234|
|                 0.0| 0.1276595744680851|
|                 0.0| 0.1702127659574468|
|0.004065040650406...|0.19148936170212766|
|0.004065040650406...|0.23404255319148937|
|0.004065040650406...| 0.2765957446808511|
|0.008130081300813009| 0.2978723404255319|
|0.012195121951219513| 0.3191489361702128|
|0.012195121951219513| 0.3617021276595745|
|0.016260162601626018| 0.3829787234042553|
|0.016260162601626018|  0.425531914893617|
| 0.02032520325203252|0.44680851063829785|
| 0.02032520325203252|0.48936170212765956|
| 0.02032520325203252| 0.5319148936170213|
|0.024390243902439025| 0.5531914893617021|
|0.028455284552845527|  0.574468085106383|
|0.036585365853658534|  0.574468085106383|
|0.044715447154471545|  0.574468085106383|
+----------

In [77]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator,BinaryClassificationEvaluator

In [55]:
my_eval = MulticlassClassificationEvaluator(labelCol="Churn",metricName="accuracy")

In [78]:
my_eval2 = BinaryClassificationEvaluator(labelCol="Churn")

In [56]:
my_eval.evaluate(results_log.predictions)

0.9112627986348123

In [79]:
my_eval2.evaluate(results_log.predictions)

0.9044282996021478

In [57]:
new_customers = spark.read.csv("new_customers.csv",inferSchema=True,header=True)
new_customers.toPandas().head()

,Names,Age,Total_Purchase,Account_Manager,Years,Num_Sites,Onboard_date,Location,Company
0,Andrew Mccall,37.0,9935.53,1,7.71,8.0,2011-08-29 18:37:54,"38612 Johnny Stravenue Nataliebury, WI 15717-8316",King Ltd
1,Michele Wright,23.0,7526.94,1,9.28,15.0,2013-07-22 18:19:54,"21083 Nicole Junction Suite 332, Youngport, ME...",Cannon-Benson
2,Jeremy Chang,65.0,100.00,1,1.00,15.0,2006-12-11 07:48:13,"085 Austin Views Lake Julialand, WY 63726-4298",Barron-Robertson
3,Megan Ferguson,32.0,6487.50,0,9.40,14.0,2016-10-28 05:32:13,"922 Wright Branch North Cynthialand, NC 64721",Sexton-Golden
4,Taylor Young,32.0,13147.71,1,10.00,8.0,2012-03-20 00:36:46,Unit 0789 Box 0734 DPO AP 39702,Wood LLC


In [60]:
from pyspark.ml.pipeline import Pipeline

In [61]:
# pipeline = Pipeline(stages=[assembler,ss,algo_log])

In [62]:
data.columns

['Age', 'Total_Purchase', 'Years', 'Num_Sites', 'Churn']

In [63]:
output2= assembler.transform(new_customers)

In [66]:
output2.select('features').show()

+--------------------+
|            features|
+--------------------+
|[37.0,9935.53,7.7...|
|[23.0,7526.94,9.2...|
|[65.0,100.0,1.0,1...|
|[32.0,6487.5,9.4,...|
|[32.0,13147.71,10...|
|[22.0,8445.26,3.4...|
+--------------------+



In [67]:

scaled2 = ss.fit(output2).transform(output2)

In [72]:
nc_final = scaled2.select("scaled_features").withColumnRenamed('scaled_features',"features")

In [74]:
nc_result = log_model.transform(nc_final)

In [75]:
nc_result

+--------------------+--------------------+--------------------+----------+
|            features|       rawPrediction|         probability|prediction|
+--------------------+--------------------+--------------------+----------+
|[2.35441249561055...|[11.9640914777838...|[0.99999363118953...|       0.0|
|[1.46355371348764...|[7.89430094558233...|[0.99962727651434...|       0.0|
|[4.13613005985637...|[8.55209850193224...|[0.99980689792496...|       0.0|
|[2.03624864485237...|[8.24569851934100...|[0.99973768405131...|       0.0|
|[2.03624864485237...|[11.5394092396996...|[0.99999026145639...|       0.0|
|[1.39992094333600...|[9.66497938966548...|[0.99993653632451...|       0.0|
+--------------------+--------------------+--------------------+----------+



# Made By Ratnakar Maurya 
#### Cleaner Code will Be updated Soon Thanks